# Visualize hash outputs


## gMiMC hash

In our current efforts towards the cryptanalysis of the generalized MiMC hash function, we are attempting to visualize the hash outputs in Cartesian coordinates.

I referenced a implementation of the hash function from [STARKWARE hash challenge](https://starkware.co/hash-challenge-implementation-reference-code/) and [GuildOfWeavers/distaff](https://github.com/GuildOfWeavers/distaff)

For our initial attempt, we are observing the visualized images of hash outputs that are produced using a small number of primes within the finite field.


In [2]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
:dep ff = { version = "0.13", features = ["derive"] }
:dep gmimc_rust_test = { package = "gmimc-rust-test", path = "../" }

In [3]:
extern crate ff;
extern crate gmimc_rust_test;

use gmimc_rust_test::gmimc::gmimc_erf;
use ff::PrimeField;

fn gmimc_test() {
    #[derive(PrimeField)]
    #[PrimeFieldModulus = "23"]
    #[PrimeFieldGenerator = "1"]
    #[PrimeFieldReprEndianness = "little"]
    struct F([u64; 1]);
    
    println!("F::ROOT_OF_UNITY: {:?}", F::ROOT_OF_UNITY);
    
    let mimc = gmimc_erf::<F> {
            capacity: 3,
            words: 2,
            round: 121,
            _field: std::marker::PhantomData::<F>,
        };

    let result = mimc.get_hash_output(&[1u128, 2, 3, 4]);
    println!("result: {:?}", result);
}

gmimc_test();

F::ROOT_OF_UNITY: F(0x0000000000000001)
result: [18, 15, 4, 20]


We can observe above that the `Root of Unity` for the prime field is '0x1'. This implementation of the finite field, `ff`, calculates the `ROOT_OF_UNITY` during initialization using the `Modulus` and `Generator`.

Here's a simple demonstration of the GMiMC hash function using `[1, 2, 3, 4]` as the input argument. This hash function is configured with four branches (capacity + 1) and undergoes 121 rounds of mutation.

To generate a result, the  GMiMCerf hash function requires four inputs. However, we'll only use one input value at the right-most end of the input array, with the remaining elements set to `0`. For instance, if we want to evaluate the hash output using 1 as the input value, the input array `X` would be `[0u128, 0, 0, 1]`.

When visualizing the output, we have two options: we can either select one of the result values or calculate the modulus of the sum of all elements of the output result.

If we choose the first option, given that the round number is 121, `result[1]` would be an appropriate choice.

For the second option, the process is equally straightforward – we simply compute `sum(result) % modulus_number`.

In [4]:
extern crate plotters;
use plotters::prelude::*;

evcxr_figure((1024, 512), |root| {
    // The following code will create a chart context
    let areas = root.split_evenly((1,2));
    let mut charts = vec![];
    
    // The following code will create a chart context
   for (area, name) in areas.iter().zip(["'result[1]'", "'sum(result) % modulus'"].into_iter()) {
        let mut chart = ChartBuilder::on(&area)
            .caption(format!("GMiMC result with {}", name), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0u128..30u128, 0u128..30u128)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .draw()?;
        charts.push(chart);
    }
    
    #[derive(PrimeField)]
    #[PrimeFieldModulus = "23"]
    #[PrimeFieldGenerator = "11"]
    #[PrimeFieldReprEndianness = "little"]
    struct F([u64; 1]);
    
    println!("F::ROOT_OF_UNITY: {:?}", F::ROOT_OF_UNITY);

    // configuration for low prime number field
    let gmimc = gmimc_erf::<F, 6> {
        capacity: 3,
        words: 2,
        round: 121,
        _field: std::marker::PhantomData::<F>,
    };

    let sequencial_points: (Vec<(u128, u128)>, Vec<(u128, u128)>) = {
        let mut points_a: [(u128, u128); 30] = [(0, 0); 30];
        let mut points_b: [(u128, u128); 30] = [(0, 0); 30];
        
        // get two types of result
        for x in 0..30 {
            let hash_output = gmimc.get_hash_output(&[0, 0, 0, x as u128]);
            let y_a = hash_output[1];
            let y_b = hash_output.clone()
                .into_iter()
                .fold(0, |acc, x| (acc + x) % 23 as u128);
            
            points_a[x] = (x as u128, y_a);
            points_b[x] = (x as u128, y_b);
        }
        (points_a.to_vec(), points_b.to_vec())
    };

    
    charts[0].draw_series(sequencial_points.0
            .iter()
            .map(|(x, y)| Circle::new((*x, *y), 1, BLACK.filled())));
    charts[1].draw_series(sequencial_points.1
            .iter()
            .map(|(x, y)| Circle::new((*x, *y), 1, BLACK.filled())));
    
    Ok(())
}).style("width:90%")

F::ROOT_OF_UNITY: F(0x0000000000000016)


GMiMC result with 'result[1]'
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
25
 
 
 
30
 
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
25
 
 
 
30
 
 
 
GMiMC result with 'sum(result) % modulus'
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
25
 
 
 
30
 
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
25
 
 
 
30

There's a different `GENERATOR` number applied to the `F` instance, but we can still expect the result to be the same as `0x1`.

Let's just try on little bit bigger prime number.

In [5]:
evcxr_figure((2048, 1024), |root| {
    // define finite field
    #[derive(PrimeField)]
    #[PrimeFieldModulus = "523"] 
    #[PrimeFieldGenerator = "2"]
    #[PrimeFieldReprEndianness = "little"]
    struct F([u64; 1]);
    
    // The following code will create a chart context
    const prime_no: u128 = 523 as u128;
    let areas = root.split_evenly((1,2));
    let mut charts = vec![];
    
   for (area, name) in areas.iter().zip(["'result[1]'", "'sum(result) % modulus'"].into_iter()) {
        let mut chart = ChartBuilder::on(&area)
            .caption(format!("GMiMC result with {}", name), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0u128..prime_no, 0u128..prime_no)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .draw()?;
        charts.push(chart);
    }

    // configuration for low prime number field
    let gmimc = gmimc_erf::<F, 6> {
        capacity: 3,
        words: 2,
        round: 121,
        _field: std::marker::PhantomData::<F>,
    };

    let sequencial_points: (Vec<(u128, u128)>, Vec<(u128, u128)>) = {
        let mut points_a: [(u128, u128); prime_no as usize] = [(0, 0); prime_no as usize];
        let mut points_b: [(u128, u128); prime_no as usize] = [(0, 0); prime_no as usize];
        
        // get two types of result
        for x in 0..prime_no as usize {
            let hash_output = gmimc.get_hash_output(&[0, 0, 0, x as u128]);
            let y_a = hash_output[1];
            let y_b = hash_output.clone()
                .into_iter()
                .fold(0, |acc, x| (acc + x) % prime_no);
            
            points_a[x] = (x as u128, y_a);
            points_b[x] = (x as u128, y_b);
        }
        (points_a.to_vec(), points_b.to_vec())
    };

    
    charts[0].draw_series(sequencial_points.0
            .iter()
            .map(|(x, y)| Circle::new((*x, *y), 1, BLACK.filled())));
    charts[1].draw_series(sequencial_points.1
            .iter()
            .map(|(x, y)| Circle::new((*x, *y), 1, BLACK.filled())));
    
    Ok(())
}).style("width:90%")

GMiMC result with 'result[1]'
 
 
 
0
 
 
 
100
 
 
 
200
 
 
 
300
 
 
 
400
 
 
 
500
 
 
 
 
0
 
 
 
100
 
 
 
200
 
 
 
300
 
 
 
400
 
 
 
500
 
 
 
GMiMC result with 'sum(result) % modulus'
 
 
 
0
 
 
 
100
 
 
 
200
 
 
 
300
 
 
 
400
 
 
 
500
 
 
 
 
0
 
 
 
100
 
 
 
200
 
 
 
300
 
 
 
400
 
 
 
500